# Exp 03: Compute the sizes of all search spaces

In [1]:
# standard libraries
from time import perf_counter_ns
import time

In [2]:
# external libraries
from matplotlib.patches import Rectangle
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd
import seaborn as sns

# custom libraries
from _run_constants import *
from part_00_file_db_utils import *
from part_00_process_functions import *

# Load Data

In [3]:
# load data
word_df, wg_df, letter_dict, char_matrix, \
    word_group_id_list, word_id_list, wchar_matrix = load_input_data(
        db_path=rc.DB_PATH, db_name=rc.DB_NAME,
        in_file_path=rc.IN_FILE_PATH)

...loading words into a dataframe...
...query execution took: 0.47 seconds...
...loading word groups into a dataframe...
...query execution took: 0.46 seconds...
...loading the letter dictionary...
...loading the char matrix...
...subsetting the char matrix...


In [4]:
# compute the sizes of all letter selectors
wg_df['n_records'] = int(1)

In [5]:
max_letter_selector_length = wg_df['letter_group_ranked'].str.len().max()
col_names = ['letter_selector_temp', 'n_records']
ls_df_list = []
for ls_nchar in range(1, max_letter_selector_length + 1):
    ls_df = build_letter_selector_df(df = wg_df, ls_nchar=ls_nchar,
                                     col_names=col_names,
                                     letter_selector_col_name='letter_selector_temp',
                                     letter_selector_id_col_name='letter_selector_temp_id')    
    ls_df_list.append(ls_df)

In [6]:
tot_ls_df = pd.concat(objs=ls_df_list,axis = 0)     
tot_ls_df.head()

,letter_selector_temp,ls_count,ls_nchar_iter,ls_nchar,letter_selector_temp_id
0,a,4,1,1,0
1,b,24368,1,1,1
2,c,3921,1,1,2
3,d,16164,1,1,3
4,e,1,1,1,4


In [7]:
tot_ls_df.shape

(1090688, 5)

In [8]:
tot_ls_df['n_records'] = int(1)

In [9]:
pv_df = pd.pivot_table(data = tot_ls_df, values = 'n_records', index = 'ls_nchar_iter',
                       columns = 'ls_nchar', aggfunc='sum', fill_value=0)

In [10]:
pv_df.head()

ls_nchar,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
ls_nchar_iter,,,,,,,,,,,,,,,,
1,26,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,26,315,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,26,111,2250,0,0,0,0,0,0,0,0,0,0,0,0,0
4,26,111,905,9180,0,0,0,0,0,0,0,0,0,0,0,0
5,26,111,905,3393,22665,0,0,0,0,0,0,0,0,0,0,0


In [11]:
pv_df.to_excel(excel_writer='check_ls_cross_tab.xlsx', index = True)

In [12]:
w_tot_ls_df = tot_ls_df.drop_duplicates(subset = ['letter_selector_temp', 'ls_count']).reset_index(drop = True)
w_tot_ls_df.shape

(250208, 6)

In [13]:
# function to return the index position of each letter
def get_ls_index(ls:str):
    return [letter_dict[l] for l in ls]

In [14]:
w_tot_ls_df['ls_index'] = w_tot_ls_df['letter_selector_temp'].map(get_ls_index)

In [15]:
w_tot_ls_df.head()

,letter_selector_temp,ls_count,ls_nchar_iter,ls_nchar,letter_selector_temp_id,n_records,ls_index
0,a,4,1,1,0,1,[0]
1,b,24368,1,1,1,1,[1]
2,c,3921,1,1,2,1,[2]
3,d,16164,1,1,3,1,[3]
4,e,1,1,1,4,1,[4]


In [16]:
# Use the GPU to the query the matrix.
# We are querying the matrix 250,208 times. 
# this will take a long time. 

In [17]:
n_search_space_list = np.zeros(shape = w_tot_ls_df.shape[0], dtype = np.int32)
for ls_row_id, ls_row in w_tot_ls_df.iloc[:None].iterrows():    
    if ls_row_id % 1000 == 0:
        print(ls_row_id)    
    
    # get letter selector id information    
    ls_id_index = np.array(ls_row['ls_index'])    

    ##
    # BUILD A COLUMN SELECTOR
    ##
    # make sure that only values GTE 0 are selected so that the right number of
    # columns are return.
    #curr_ls_id = ls_id_index[ls_id_index >= 0]
    
    ##
    # SUBSET THE wchar_matrix by column selector
    ##    
    letter_selector_count = np.all(wchar_matrix[:, ls_id_index] >= 1, axis=1).sum()
    n_search_space_list[ls_row_id] = letter_selector_count    
    
    # this is the number of word groups that meet certain criteria. 
    # for example, words that feature the letters: 'bro'    
    #n_search_space = temp_wg_id_list.shape[0]
    

#tot_ls_df['n_search_space'] = n_search_space_list

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [18]:
w_tot_ls_df['n_search_space'] = n_search_space_list

In [19]:
tot_ls_df.head()

,letter_selector_temp,ls_count,ls_nchar_iter,ls_nchar,letter_selector_temp_id,n_records
0,a,4,1,1,0,1
1,b,24368,1,1,1,1
2,c,3921,1,1,2,1
3,d,16164,1,1,3,1
4,e,1,1,1,4,1


In [20]:
w_tot_ls_df.head()

,letter_selector_temp,ls_count,ls_nchar_iter,ls_nchar,letter_selector_temp_id,n_records,ls_index,n_search_space
0,a,4,1,1,0,1,[0],133001
1,b,24368,1,1,1,1,[1],35319
2,c,3921,1,1,2,1,[2],80811
3,d,16164,1,1,3,1,[3],56705
4,e,1,1,1,4,1,[4],145106


In [21]:
# join back with the total df
testo = pd.merge(left = tot_ls_df, right = w_tot_ls_df[['letter_selector_temp', 'ls_count',  'ls_index', 'n_search_space', ]])
testo.shape

(1090688, 8)

In [22]:
tot_ls_df = testo.copy()

In [23]:
tot_ls_df.shape

(1090688, 8)

In [24]:
tot_ls_df.head()

,letter_selector_temp,ls_count,ls_nchar_iter,ls_nchar,letter_selector_temp_id,n_records,ls_index,n_search_space
0,a,4,1,1,0,1,[0],133001
1,b,24368,1,1,1,1,[1],35319
2,c,3921,1,1,2,1,[2],80811
3,d,16164,1,1,3,1,[3],56705
4,e,1,1,1,4,1,[4],145106


In [25]:
tot_ls_df['ls_index'] = tot_ls_df['ls_index'].astype(str)

In [26]:
write_data_to_sqlite(df = tot_ls_df, table_name='exp_02_search_space_size',
                     db_path = rc.DB_PATH, db_name = rc.DB_NAME)

...now writing: exp_02_search_space_size


In [27]:
# use CuPy to query the matrix.

In [28]:
# try the above with cupy!
import cupy as cp

In [29]:
wchar_matrix_cp = cp.asarray(a=wchar_matrix)

n_search_space_list = cp.zeros(shape = w_tot_ls_df.shape[0], dtype = int)
for ls_row_id, ls_row in w_tot_ls_df.iloc[:None].iterrows():    
    if ls_row_id % 1000 == 0:
        print(ls_row_id)    
    
    # get letter selector id information    
    ls_id_index = cp.asarray(ls_row['ls_index'])    
        
    ##
    # SUBSET THE wchar_matrix by column selector
    ##    
    n_search_space_list[ls_row_id] = cp.all(wchar_matrix_cp[:, ls_id_index] >= 1, axis=1).sum()
    
    # this is the number of word groups that meet certain criteria. 
    # for example, words that feature the letters: 'bro'

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [30]:
# much faster on the GPU!